<a href="https://colab.research.google.com/github/theakash07/Machine_learning/blob/main/Insectclassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Cloning YOLO github
!git clone https://github.com/theakash07/yolov9
!pip install -r yolov9/requirements.txt
!pip install utils
!pip install pylabel > /dev/null

Cloning into 'yolov9'...
remote: Enumerating objects: 390, done.
remote: Counting objects: 100% (232/232), done.
remote: Compressing objects: 100% (117/117), done.
remote: Total 390 (delta 179), reused 115 (delta 115), pack-reused 158
Receiving objects: 100% (390/390), 2.56 MiB | 15.25 MiB/s, done.
Resolving deltas: 100% (197/197), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 34.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 43.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.4 MB/s eta 0:00:00
     ━━━━━

In [2]:
import torch
import os
import random
import shutil
import xml.etree.ElementTree as ET
import numpy as np
import matplotlib.pyplot as plt
import logging
import os
import zipfile
import yaml
import PIL

from IPython.display import Image  # for displaying images
from sklearn.model_selection import train_test_split
from pylabel import importer
from xml.dom import minidom
from tqdm import tqdm
from PIL import Image, ImageDraw

logging.getLogger().setLevel(logging.CRITICAL)
random.seed(42)

In [3]:
from pylabel import importer

In [4]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [7]:
!cp -r "/content/drive/My Drive/InsectAi reference paper/archive/ImageClassesCombinedWithCOCOAnnotations/images_raw" ./

In [8]:
!cp -r "/content/drive/MyDrive/InsectAi reference paper/archive/ImageClassesCombinedWithCOCOAnnotations/coco_instances.json" ./

In [9]:
!cp -r "/content/drive/MyDrive/InsectAi reference paper/archive/ImageClassesCombinedWithCOCOAnnotations/test_image.jpg" ./

In [10]:
path_to_annotations = r"./coco_instances.json"

In [11]:
path_to_images = r"./images_raw"

In [12]:
dataset = importer.ImportCoco(path_to_annotations, path_to_images=path_to_images, name="BCCD_coco")
dataset.df.head(5)

,img_folder,img_filename,img_path,img_id,img_width,img_height,img_depth,ann_segmented,ann_bbox_xmin,ann_bbox_ymin,...,ann_iscrowd,ann_keypoints,ann_pose,ann_truncated,ann_difficult,cat_id,cat_name,cat_supercategory,split,annotated
id,,,,,,,,,,,,,,,,,,,,,
0,./images_raw,00000000.jpg,,0,300,300,,,104.5,114.5,...,0,,,,,4,dragonfly,insect,,1
1,./images_raw,00000001.jpg,,1,300,300,,,74.5,49.5,...,0,,,,,7,scorpion,insect,,1
2,./images_raw,00000002.jpg,,2,300,300,,,67.5,87.5,...,0,,,,,3,cicada,insect,,1
3,./images_raw,00000002.jpg,,2,300,300,,,128.5,119.0,...,0,,,,,6,snail,insect,,1
4,./images_raw,00000002.jpg,,2,300,300,,,162.5,84.5,...,0,,,,,10,beetle,insect,,1


Analyze annotations
Pylabel can calculate basic summary statisticts about the dataset such as the number of files and the classes. The dataset is stored as a pandas frame so the developer can do additional exploratory analysis on the dataset.

In [14]:
print(f"Number of images: {dataset.analyze.num_images}")
print(f"Number of classes: {dataset.analyze.num_classes}")
print(f"Classes:{dataset.analyze.classes}")
print(f"Class counts:\n{dataset.analyze.class_counts}")
print(f"Path to annotations:\n{dataset.path_to_annotations}")

Number of images: 15000
Number of classes: 10
Classes:['grasshoppers', 'butterfly', 'cicada', 'dragonfly', 'spider', 'snail', 'scorpion', 'bees', 'moth', 'beetle']
Class counts:
grasshoppers    3043
dragonfly       3042
beetle          3028
butterfly       3014
spider          3010
cicada          3004
snail           2973
scorpion        2954
bees            2951
moth            2928
Name: cat_name, dtype: int64
Path to annotations:
.
